## Import necessary libraries

In [1]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

import gensim
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models
from gensim.models import LsiModel

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings('ignore')

/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


## Read cleaned data files

### Read cleaned jobs listings file and save in dataframe

In [2]:
cleaned_jobs_listings_df = pd.read_csv("cleaned_jobs_listings.csv")

### Check columns info in jobs listings dataframe

In [3]:
cleaned_jobs_listings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3690 entries, 0 to 3689
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   job_title                3690 non-null   object 
 1   salary_estimate          3690 non-null   object 
 2   job_description          3690 non-null   object 
 3   rating                   3690 non-null   float64
 4   company_name             3690 non-null   object 
 5   location                 3690 non-null   object 
 6   headquarters             3690 non-null   object 
 7   size                     3690 non-null   object 
 8   founded                  3690 non-null   int64  
 9   type_of_ownership        3690 non-null   object 
 10  industry                 3690 non-null   object 
 11  sector                   3690 non-null   object 
 12  revenue                  3690 non-null   object 
 13  competitors              3690 non-null   object 
 14  easy_apply              

## Data Preprocessing

### Define stopwords to append to default stopwords

In [4]:
# append special stopwords in articles to default and common stopwords
new_stopwords = STOPWORDS.union(set(['job', 'skill', 'experience', 'team', 'data', 'use', 'like', 'business', 'work', 'ability', 'let', 'example'
                'need', 'new', 'user', 'opportunity', 'candidate', 'provide', 'company', 'one', 'used', 'need', 
                'see', 'make', 'follow', 'going', 'will', 'want', 'well', 'find', 'give', 'change', 'look', 'first', 'using',
                'know', 'model', 'science', 'think', 'year', 'years', 'looking']))

### Method to preprocess data for articles

In [5]:
# lemmatize = words change to noun
# stemmed = words reduced to root form
def lemmatize(text):
    return WordNetLemmatizer().lemmatize(text, pos='n')

# Convert a document into a list of tokens.
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in new_stopwords:
            result.append(lemmatize(token))
    return result

In [6]:
preprocessed_jobs_listings = cleaned_jobs_listings_df['cleaned_job_description'].fillna('').astype(str).map(preprocess)

In [7]:
# implements the concept of a Dictionary – a mapping between words and their integer ids.
jobs_listings_dictionary = corpora.Dictionary(preprocessed_jobs_listings)

In [8]:
# Convert into the bag-of-words (BoW) format
jobs_listings_bow_corpus = [jobs_listings_dictionary.doc2bow(text) for text in preprocessed_jobs_listings]

In [9]:
# Convert into TF-IDF format
jobs_listings_tfidf = models.TfidfModel(jobs_listings_bow_corpus)
jobs_listings_tfidf_corpus = jobs_listings_tfidf[jobs_listings_bow_corpus]

## LSA Model

### Result using LSA model + Bag of words

In [10]:
jobs_listings_bow_lsamodel = LsiModel(jobs_listings_bow_corpus, num_topics=10, id2word = jobs_listings_dictionary)

In [11]:
jobs_listings_bow_lsamodel.print_topics(num_words=7)

[(0,
  '0.173*"development" + 0.164*"analysis" + 0.151*"skill" + 0.143*"product" + 0.141*"technology" + 0.138*"analytics" + 0.134*"information"'),
 (1,
  '0.246*"learning" + 0.217*"analytics" + -0.205*"information" + 0.199*"machine" + -0.168*"laboratory" + -0.165*"research" + -0.160*"employee"'),
 (2,
  '0.397*"learning" + 0.319*"machine" + -0.179*"management" + -0.174*"process" + 0.168*"research" + 0.161*"employee" + -0.141*"project"'),
 (3,
  '0.351*"research" + -0.269*"information" + -0.193*"application" + 0.192*"scientist" + 0.192*"laboratory" + 0.191*"product" + 0.171*"clinical"'),
 (4,
  '-0.383*"development" + 0.327*"analysis" + 0.280*"analytics" + -0.224*"technology" + 0.193*"statistical" + -0.141*"product" + 0.137*"insight"'),
 (5,
  '-0.336*"research" + 0.302*"product" + -0.224*"learning" + 0.196*"status" + -0.180*"machine" + 0.177*"people" + 0.154*"employment"'),
 (6,
  '-0.358*"laboratory" + 0.266*"product" + 0.229*"research" + -0.225*"service" + -0.217*"medical" + -0.176*"

### Result using LSA model + TF-IDF

In [12]:
jobs_listings_tfidf_lsamodel = LsiModel(jobs_listings_tfidf_corpus, num_topics=10, id2word = jobs_listings_dictionary)

In [13]:
jobs_listings_tfidf_lsamodel.print_topics(num_words=7)

[(0,
  '0.098*"learning" + 0.096*"analytics" + 0.093*"machine" + 0.086*"product" + 0.076*"research" + 0.076*"solution" + 0.076*"customer"'),
 (1,
  '0.258*"laboratory" + 0.168*"clinical" + 0.154*"cell" + -0.125*"machine" + -0.123*"learning" + 0.112*"assay" + -0.110*"aws"'),
 (2,
  '0.177*"analyze" + 0.171*"locate" + 0.163*"printout" + 0.159*"databasesdata" + 0.155*"database" + 0.145*"qualityacquire" + 0.145*"setsfilter"'),
 (3,
  '-0.152*"university" + 0.135*"cell" + -0.129*"security" + -0.123*"clery" + -0.111*"investigation" + 0.104*"learning" + 0.100*"machine"'),
 (4,
  '0.723*"tutor" + 0.485*"tutoring" + 0.177*"student" + 0.129*"choose" + 0.123*"session" + 0.088*"schedule" + 0.081*"tutoringonline"'),
 (5,
  '-0.156*"aws" + -0.149*"laboratory" + -0.142*"cell" + -0.133*"pipeline" + -0.127*"cloud" + 0.116*"university" + -0.112*"spark"'),
 (6,
  '-0.190*"machine" + -0.189*"learning" + -0.168*"cell" + -0.132*"ml" + -0.110*"aws" + -0.110*"algorithm" + -0.108*"biology"'),
 (7,
  '-0.299*"l

## LDA Model

### Result using LDA model + Bag of words

In [14]:
jobs_listings_bow_ldamodel = gensim.models.LdaMulticore(jobs_listings_bow_corpus, num_topics=10, id2word=jobs_listings_dictionary)

In [15]:
jobs_listings_bow_ldamodel.print_topics(num_words=7)

[(0,
  '0.007*"product" + 0.007*"skill" + 0.006*"analytics" + 0.005*"analysis" + 0.005*"learning" + 0.004*"related" + 0.004*"tool"'),
 (1,
  '0.005*"product" + 0.005*"learning" + 0.005*"analysis" + 0.004*"technology" + 0.004*"service" + 0.004*"including" + 0.004*"design"'),
 (2,
  '0.005*"analytics" + 0.005*"technology" + 0.005*"engineering" + 0.005*"design" + 0.004*"product" + 0.004*"skill" + 0.004*"analysis"'),
 (3,
  '0.007*"analysis" + 0.006*"solution" + 0.006*"project" + 0.006*"knowledge" + 0.006*"development" + 0.005*"support" + 0.005*"research"'),
 (4,
  '0.006*"technology" + 0.005*"service" + 0.005*"learning" + 0.005*"analysis" + 0.005*"product" + 0.004*"solution" + 0.004*"development"'),
 (5,
  '0.009*"development" + 0.005*"including" + 0.005*"analysis" + 0.005*"technology" + 0.005*"scientist" + 0.004*"working" + 0.004*"project"'),
 (6,
  '0.005*"skill" + 0.005*"learning" + 0.004*"analysis" + 0.004*"working" + 0.004*"product" + 0.004*"application" + 0.004*"information"'),
 (7,

In [16]:
pyLDAvis.gensim_models.prepare(jobs_listings_bow_ldamodel, jobs_listings_bow_corpus, jobs_listings_dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.006952  0.005028       1        1  22.616877
5      0.010432 -0.010522       2        1  20.528102
3      0.008974  0.005540       3        1  15.602592
4     -0.007376  0.002945       4        1   8.315537
6     -0.006938  0.001392       5        1   7.299666
8     -0.004224 -0.007880       6        1   7.021944
9      0.002532  0.007342       7        1   6.561982
7      0.000559 -0.000040       8        1   4.162719
1     -0.011820 -0.000635       9        1   4.116522
2      0.000911 -0.003171      10        1   3.774060, topic_info=            Term         Freq        Total Category  logprob  loglift
61   engineering  2699.000000  2699.000000  Default  30.0000  30.0000
205    analytics  3819.000000  3819.000000  Default  29.0000  29.0000
307   technology  3999.000000  3999.000000  Default  28.0000  28.0000
98      learning  3758.000000  3758.000000  Default  27.0000  27.0000
134      product  4363.000000  4363.000000  Default  26.0000  26.0000
..           ...          ...          ...      ...      ...      ...
428      process   108.079986  3328.462019  Topic10  -5.7866  -0.1504
286      problem    90.731331  2264.117572  Topic10  -5.9615   0.0600
517  information   101.934037  3257.410032  Topic10  -5.8451  -0.1873
50   development   115.098118  5144.443619  Topic10  -5.7237  -0.5229
98      learning    98.450139  3758.553630  Topic10  -5.8799  -0.3652

[864 rows x 6 columns], token_table=       Topic      Freq           Term
term                                 
38482      1  0.434069  abilitiesliag
38482     10  0.434069  abilitiesliag
0          1  0.225269         access
0          2  0.174584         access
0          3  0.186973         access
...      ...       ...            ...
18751      6  0.608981            年收入
18752      6  0.613952          年起共累计
18753      6  0.611374          日前暂停h
18754      6  0.614248      毕业生以及正在寻找
18755      6  0.608924     特朗普zf也出台新政

[2285 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 6, 4, 5, 7, 9, 10, 8, 2, 3])

### Result using LDA model + TF-IDF

In [17]:
jobs_listings_tfidf_ldamodel = gensim.models.LdaMulticore(jobs_listings_tfidf_corpus, num_topics=10, id2word=jobs_listings_dictionary)

In [26]:
jobs_listings_tfidf_ldamodel.print_topics(num_words=9)

[(0,
  '0.001*"learning" + 0.001*"machine" + 0.001*"product" + 0.001*"client" + 0.001*"aws" + 0.001*"research" + 0.001*"statistical" + 0.001*"engineer" + 0.001*"model"'),
 (1,
  '0.001*"learning" + 0.001*"analytics" + 0.001*"machine" + 0.001*"product" + 0.001*"cloud" + 0.001*"engineer" + 0.001*"model" + 0.001*"platform" + 0.001*"modeling"'),
 (2,
  '0.001*"analytics" + 0.001*"learning" + 0.001*"analyst" + 0.001*"model" + 0.001*"marketing" + 0.001*"customer" + 0.001*"database" + 0.001*"client" + 0.000*"product"'),
 (3,
  '0.001*"learning" + 0.001*"machine" + 0.001*"analytics" + 0.001*"research" + 0.000*"client" + 0.000*"product" + 0.000*"insight" + 0.000*"solution" + 0.000*"model"'),
 (4,
  '0.001*"learning" + 0.001*"cell" + 0.001*"product" + 0.001*"research" + 0.001*"information" + 0.001*"analytics" + 0.001*"client" + 0.001*"machine" + 0.001*"clinical"'),
 (5,
  '0.001*"product" + 0.001*"analytics" + 0.001*"research" + 0.001*"database" + 0.001*"solution" + 0.001*"requirement" + 0.001*"

In [19]:
dominant_topics_df = pd.DataFrame()
for i, row in enumerate(jobs_listings_tfidf_ldamodel[jobs_listings_tfidf_corpus]):
   row = sorted(row, key=lambda x: (x[1]), reverse=True)
   for j, (topic_num, prop_topic) in enumerate(row):
        if j == 0: # => dominant topic
            wp = jobs_listings_tfidf_ldamodel.show_topic(topic_num)
            topic_keywords = ", ".join([word for word, prop in wp])
            dominant_topics_df = dominant_topics_df.append(
                pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True
            )
        else:
          break
dominant_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

In [20]:
dominant_topics_df["Text"] = cleaned_jobs_listings_df['cleaned_job_description']
dominant_topics_df

,Dominant_Topic,Perc_Contribution,Topic_Keywords,Text
0,5.0,0.9214,"product, analytics, research, database, soluti...",about at were on a mission to make booking tr...
1,8.0,0.9202,"learning, database, analyst, statistical, anal...",at noom we use scientifically proven methods t...
2,9.0,0.9234,"learning, machine, product, research, analytic...",decodemhttpswwwdecodemcomdata science manager ...
3,8.0,0.9133,"learning, database, analyst, statistical, anal...",seeks a dynamic and driven midlevel data anal...
4,3.0,0.9200,"learning, machine, analytics, research, client...",director data science 200537descriptionedelma...
...,...,...,...,...
3685,6.0,0.5685,"product, analytics, learning, pipeline, machin...",is a leading staffing and recruitment company...
3686,0.0,0.9098,"learning, machine, product, client, aws, resea...",about us is a digital transformation consultin...
3687,7.0,0.9347,"learning, machine, analytics, model, research,...",job descriptionthe security analytics data eng...
3688,7.0,0.9254,"learning, machine, analytics, model, research,...",the security analytics data engineer will inte...


In [21]:
dominant_topics_df = dominant_topics_df.sort_values(by=['Perc_Contribution'], ascending=False)
dominant_topics_df.head(50)

,Dominant_Topic,Perc_Contribution,Topic_Keywords,Text
1141,7.0,0.9489,"learning, machine, analytics, model, research,...",work shift daywork week variesjob summarythe m...
289,7.0,0.9486,"learning, machine, analytics, model, research,...",senior data scientist description the senior d...
3359,6.0,0.9486,"product, analytics, learning, pipeline, machin...",we are hiringdata engineer fulltime day aust...
1234,7.0,0.9486,"learning, machine, analytics, model, research,...",work shift daywork week m f w occasional week...
3287,5.0,0.9486,"product, analytics, research, database, soluti...",location pickle research campusjob posting tit...
3261,6.0,0.9484,"product, analytics, learning, pipeline, machin...",why join technologies leverages technology to...
1208,7.0,0.9482,"learning, machine, analytics, model, research,...",about usharris health is a nationally recogniz...
3362,5.0,0.9479,"product, analytics, research, database, soluti...",job posting titledevopsbuild and release engin...
1233,7.0,0.9478,"learning, machine, analytics, model, research,...",work shift daywork week m f w occasional week...
1243,5.0,0.9474,"product, analytics, research, database, soluti...",this will be a remote positionjob descriptiona...


In [22]:
dominant_topics_df["Dominant_Topic"].unique()

array([7., 6., 5., 1., 0., 8., 4., 9., 2., 3.])

In [23]:
pyLDAvis.gensim_models.prepare(jobs_listings_tfidf_ldamodel, jobs_listings_bow_corpus, jobs_listings_dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.013067  0.016986       1        1  41.339910
7     -0.016756 -0.013093       2        1  29.306828
6     -0.002117  0.006982       3        1   8.769964
0     -0.004931 -0.001706       4        1   6.443072
1     -0.001501 -0.007933       5        1   3.920721
8      0.004012 -0.001648       6        1   3.266632
4      0.005910  0.000366       7        1   2.502388
2      0.008487 -0.000356       8        1   1.791726
9      0.008097  0.000068       9        1   1.787802
3      0.011866  0.000333      10        1   0.870957, topic_info=            Term        Freq       Total Category  logprob  loglift
98      learning  748.000000  748.000000  Default  30.0000  30.0000
205    analytics  706.000000  706.000000  Default  29.0000  29.0000
103      machine  710.000000  710.000000  Default  28.0000  28.0000
134      product  680.000000  680.000000  Default  27.0000  27.0000
144     research  639.000000  639.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
8       analysis    3.385657  548.722419  Topic10  -7.7836  -0.3447
135      project    3.225107  510.847825  Topic10  -7.8322  -0.3218
165  statistical    3.206042  514.517588  Topic10  -7.8381  -0.3349
161          sql    3.124256  471.693839  Topic10  -7.8639  -0.2738
307   technology    3.162041  520.396118  Topic10  -7.8519  -0.3600

[793 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
32241      1  0.316222  abinitio
32241      2  0.316222  abinitio
32241      3  0.079055  abinitio
32241      4  0.079055  abinitio
32241      7  0.158111  abinitio
...      ...       ...       ...
7642       1  0.387240   âshould
7642       2  0.290430   âshould
7642       3  0.096810   âshould
7642       4  0.096810   âshould
7642      10  0.096810   âshould

[3332 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 8, 7, 1, 2, 9, 5, 3, 10, 4])